# Srock Strategy - Single Factor Stock Selection 0001

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2015-12-31'
ed = '2026-02-13'

In [3]:
sql_factor =  f"""
SELECT
    date,
    instrument,
    dividend_yield_ratio AS factor
FROM cn_stock_prefactors
WHERE instrument NOT LIKE '%BJ%'
QUALIFY c_pct_rank(total_market_cap) > 0.20
AND c_pct_rank(pe_ttm) < 0.40
AND pe_ttm > 0 
AND ps_ttm < 2.5
"""

In [4]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 10
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
    WHERE is_month_end_trade = 1
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [5]:
df_strategy = dai.query(sql_trade, filters={"date":[sd, ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2015-12-31,000612.SZ,0.0,9.0,0.058824
1,2015-12-31,300118.SZ,0.0,9.0,0.058824
2,2015-12-31,002047.SZ,0.0,9.0,0.058824
3,2015-12-31,600688.SH,0.0,9.0,0.058824
4,2015-12-31,600133.SH,0.0,9.0,0.058824
...,...,...,...,...,...
169,2026-01-30,002011.SZ,0.0,6.5,0.083333
170,2026-01-30,600780.SH,0.0,6.5,0.083333
171,2026-01-30,002668.SZ,0.0,6.5,0.083333
172,2026-01-30,603363.SH,0.0,6.5,0.083333


## 3. Backtest Strategy

In [6]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""close""",
    order_price_field_sell="""close""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 19:04:56] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 19:04:56] [info     ] 2015-12-31, 2026-01-30, , , instruments=55
[2026-02-18 19:04:57] [info     ] bigtrader module V2.2.0
[2026-02-18 19:04:57] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
order_target_percent: price is null for '600133.SH' at '2015-12-31 15:00:00'
order_target_percent: price is null for '000626.SZ' at '2015-12-31 15:00:00'
order_target_percent: price is null for '000767.SZ' at '2015-12-31 15:00:00'
[2026-02-18 19:05:01] [info     ] backtest done, raw_perf_ds:dai.DataSource("_db5d2a7ba20c4cf38559088e1abac123")


[2026-02-18 19:05:02] [info     ] bigtrader.v34 运行完成 [5.611s].
